In [4]:
import psycopg2

# create a connection to the database

In [6]:
try:
    conn = psycopg2.connect("host= 127.0.0.1 dbname = postgres user= postgres password = Mampi93*#")
except psycopg2.Error as e:
    print("Error: coulnd not make connection tot the postgres database")
    print(e)

Error: coulnd not make connection tot the postgres database
connection to server at "127.0.0.1", port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?

